In [1]:
#### Importar librerias 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import glob
from PIL import Image
warnings.filterwarnings('ignore')

#### Definir directorio
os.chdir('../')

In [2]:
os.getcwd()

'c:\\Users\\aleja\\Documents\\Pry_ADySP\\Presidentes_Peru_image_recognition'

In [3]:
#### Procesa imagenes para homolagar 
def get_images(path):
    images = []
    for img_path in glob.glob(path): 
        img = Image.open(img_path).convert('RGB')
        img = img.resize((250,250))
        img = np.array(img)
        images.append(img)
    return np.array(images)    

In [5]:
#### Lista de presidentes de la ultima decada 
presidentes = ['1.Valentin Paniagua', '2.Alejandro Toledo', '3.Alan Garcia', '4.Ollanta Humala', '5.Pedro Pablo Kuczynski', '6.Martin Vizcarra', '7.Manuel Merino', '8.Francisco Sagasti', '9.Pedro Castillo', '10.Dina Boluarte']

all_images = []

for presidentes in presidentes :
    images = get_images(f'data_set/{presidentes}/*')
    all_images.append(images)
    print(f'End processing: {presidentes}')

End processing: 1.Valentin Paniagua
End processing: 2.Alejandro Toledo
End processing: 3.Alan Garcia
End processing: 4.Ollanta Humala
End processing: 5.Pedro Pablo Kuczynski
End processing: 6.Martin Vizcarra
End processing: 7.Manuel Merino
End processing: 8.Francisco Sagasti
End processing: 9.Pedro Castillo
End processing: 10.Dina Boluarte


In [6]:
#### Creamos las clases para cada presidente 
images = []
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
y = []

for class_ in classes:
    for i in range(0, len(all_images[class_])):
        images.append(all_images[class_][i,:,:,:])
        y.append(class_)

In [7]:
#### separamos la data para entrenamiento
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, y, test_size=0.2, random_state=122)


In [8]:
from tensorflow.keras.utils import to_categorical
x_train_esc = np.array(x_train)/255
x_test_esc = np.array(x_test)/255

y_train_cat = to_categorical(y_train)

In [12]:
print(x_test_esc.shape)

(248, 250, 250, 3)


In [16]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),  # Puedes agregar una capa densa intermedia para más capacidad
    Dense(10, activation='softmax')
])


model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'categorical_crossentropy',  ### Modelos multiclass
              metrics = ['accuracy'])

In [17]:
#### entrenar fit

model.fit(x_train_esc, y_train_cat, epochs= 10, batch_size =25, validation_split = 0.2)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 228ms/step - accuracy: 0.1399 - loss: 3.1923 - val_accuracy: 0.1616 - val_loss: 2.2667
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 222ms/step - accuracy: 0.2027 - loss: 2.1978 - val_accuracy: 0.2475 - val_loss: 2.0620
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 214ms/step - accuracy: 0.4124 - loss: 1.6810 - val_accuracy: 0.3384 - val_loss: 2.1932
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 216ms/step - accuracy: 0.8034 - loss: 0.7215 - val_accuracy: 0.3485 - val_loss: 2.8440
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 215ms/step - accuracy: 0.9572 - loss: 0.1943 - val_accuracy: 0.3283 - val_loss: 4.9078
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 215ms/step - accuracy: 0.9926 - loss: 0.0296 - val_accuracy: 0.3586 - val_loss: 5.4804
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 216ms/step - accuracy: 0.9982 - loss: 0.0085 - val_accuracy: 0.3434 - val_loss: 4.4538
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 213ms/step - accuracy: 0.9979 - loss: 0.0178 - val_accuracy: 0.

In [18]:
preds = model.predict(x_train_esc)

31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step
